In [1]:
import gym,torch,torch.nn.functional as F,numpy as np,matplotlib.pyplot as plt,rl_utils
from tqdm import tqdm

In [2]:
class PolicyNet(torch.nn.Module):
    def __init__(self,state_dim,hidden_dim,action_dim):
        super(PolicyNet,self).__init__()
        # define input layer
        self.fc1=torch.nn.Linear(state_dim,hidden_dim)
        # define output layer
        self.fc2=torch.nn.Linear(hidden_dim,action_dim)

    def forward(self, x):
        # define forward pass
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x),dim=1)

In [3]:
class ValueNet(torch.nn.Module):
    def __init__(self,state_dim,hidden_dim):
        super(ValueNet,self).__init__()
        # define input layer
        self.fc1=torch.nn.Linear(state_dim,hidden_dim)
        # define output layer
        self.fc2=torch.nn.Linear(hidden_dim,1)

    def forward(self, x):
        # define forward pass
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [4]:
class ActorCritic:
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr,critic_lr, gamma, device):
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim).to(device)# 初始化策略网络
        self.critic = ValueNet(state_dim, hidden_dim).to(device)# 初始化价值网络
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=actor_lr)# 初始化优化器
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=critic_lr)# 初始化优化器
               
        self.gamma = gamma
        self.device = device

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def update(self, transitions_dict):
        # 从字典中取出所有的训练数据
        states = torch.tensor(transitions_dict['states'], dtype=torch.float).to(self.device)
        actions = torch.tensor(transitions_dict['actions']).view(-1, 1).to(self.device)
        rewards = torch.tensor(transitions_dict['rewards']).view(-1, 1).to(self.device)
        next_states = torch.tensor(transitions_dict['next_states'], dtype=torch.float).to(self.device)
        dones = torch.tensor(transitions_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device)

        # 时序差分目标
        td_target = rewards + self.gamma * self.critic(next_states) * (1 - dones)
        td_delta = td_target - self.critic(states)
        log_probs = torch.log(self.actor(states)).gather(1, actions)
        actor_loss = torch.mean(-log_probs * td_delta.detach())# detach()防止梯度传播
        # 均方误差损失函数
        critic_loss = torch.mean(F.mse_loss(self.critic(states), td_target.detach()))
        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        self.actor_optimizer.step()
        self.critic_optimizer.step()

In [5]:
actor_lr = 1e-3
critic_lr = 1e-2
num_episodes = 1000
hidden_dim = 128
gamma = 0.98
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = 'CartPole-v0'
env = gym.make(env_name)
env.seed(0)
torch.manual_seed(0)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = ActorCritic(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, gamma, device)

return_list = rl_utils.train_on_policy_agent(env, agent, num_episodes)


c:\vnstudio\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "


AttributeError: 'CartPoleEnv' object has no attribute 'seed'

In [6]:
episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('REINFORCE on {}'.format(env_name))
plt.show()

mv_return = rl_utils.moving_average(return_list, 9)
plt.plot(episodes_list, mv_return)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('REINFORCE on {}'.format(env_name))
plt.show()

NameError: name 'return_list' is not defined